<table  align="left" width="100%"> <tr>
        <td  style="background-color:#ffffff;"><a href="https://qworld.net" target="_blank"><img src="../images/qworld.jpg" width="35%" align="left"></a></td>
        <td  align="right" style="background-color:#ffffff;vertical-align:bottom;horizontal-align:right">
            prepared by <a href="https://iitis.pl/pl/person/aglos" target="_blank"  >Adam Glos</a> and Özlem Salehi
        </td>        
</tr></table>

<table width="100%"><tr><td style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;text-align:right;">This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros. </td></tr></table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $

# <font color="blue"> Solutions for </font> Grover Algorithm for Max-Cut Problem

<a id="task1"> </a>

### Task 1

Implement the Grover algorithm for the following graph
<img src="../images/completebipartite.png" width="25%" align="center">

and check whether there is a coloring in which there are exactly four edges connecting vertices with a different color. Apply the oracle twice.

<h3> Solution </h3>

In [ ]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

# 0-3: vertices
# 4-7: edge checking
# 8-10: the number
# 11: auxillary

def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

def oracle_computation():
    # check all edges
    yield edge_check(0, 2, 4)
    yield edge_check(0, 3, 5)
    yield edge_check(1, 2, 6)
    yield edge_check(1, 3, 7)
    
    # add outputs of edge checking
    yield CX(qq[4],qq[8])
    
    yield CCX(qq[5], qq[8], qq[9])
    yield CX(qq[5],qq[8])
    
    yield CCX(qq[6], qq[8], qq[9])
    yield CX(qq[6],qq[8])
    
    yield X(qq[10]).controlled_by(qq[7], qq[8],qq[9])
    yield CCX(qq[7], qq[8], qq[9])
    yield CX(qq[7],qq[8])
    
    
    # check if number is equal to four 
    yield X(qq[8])
    yield X(qq[9])
    yield X(qq[11]).controlled_by(*(qq[8:11]))
    # note we don't have to undo the X gates!
    
def oracle():    
    yield oracle_computation()
    yield Z(qq[11])
    yield inverse(oracle_computation())
    
def grover_diffusion():
    yield H.on_each(*(qq[:4]))
    yield X.on_each(*(qq[:4]))
    yield Z(qq[3]).controlled_by(*(qq[:3]))
    yield X.on_each(*(qq[:4]))
    yield H.on_each(*(qq[:4]))

# the Grover algorithm
qq = cirq.LineQubit.range(12) 
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:4])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
circuit.append(cirq.measure(*qq[0:4], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**4)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

We observe that the probability of observing either "1100" and "0011" is around 95%. These two colorings correspond to the case where the nodes 0 and 1 are colored using the first color, and 2 and 3 are colored using the second color.  

<a id="task2"></a>
### Task 2

Implement the Grover Algorithm for the following graph
<img src="../images/finalgrover1.png" width="25%" align="center">

and check whether there is coloring with six or more edges connecting vertices with a different color. Apply the oracle twice. 

*Hint*: Note that you have to check only two bits of the number.

<h3> Solution </h3>

In [ ]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

# 0-4: vertices
# 5-11: edge checking
# 12-14: the number
# 15: auxillary

def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

def oracle_computation():
    # check all edges
    yield edge_check(0, 1, 5)
    yield edge_check(0, 3, 6)
    yield edge_check(0, 4, 7)
    yield edge_check(1, 3, 8)
    yield edge_check(1, 4, 9)
    yield edge_check(2, 3, 10)
    yield edge_check(2, 4, 11)
    
    # add qubit 5   
    yield CX(qq[5], qq[12])
    
    # add qubits 6-7
    for j in range(6,8):
        yield CCX(qq[j], qq[12], qq[13])
        yield CX(qq[j], qq[12])
    
    # add qubits 8-11
    for j in range(8,12):
        yield X(qq[14]).controlled_by(qq[j], qq[12], qq[13])
        yield CCX(qq[j], qq[12], qq[13])
        yield CX(qq[j], qq[12])

    # set the last bit 
    # we do not have to check 12th qubit
    yield CCX(qq[13], qq[14], qq[15])
    
def oracle():    
    yield oracle_computation()
    yield Z(qq[15])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*(qq[:5]))
    yield X.on_each(*(qq[:5]))
    yield Z(qq[4]).controlled_by(*(qq[:4]))
    yield X.on_each(*(qq[:5]))
    yield H.on_each(*(qq[:5]))


# the Grover algorithm
qq = cirq.LineQubit.range(16) 
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:5])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
       
circuit.append(cirq.measure(*qq[0:5], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**5)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)